In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()

In [6]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# View all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [9]:
# Save references to each table
measurement = Base.classes.measurement
station= Base.classes.station

In [10]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [12]:
# Use the session to query measurement table and display the first 5 stations
for row in session.query(measurement, measurement.station).limit(5).all():
    print(row)

(<sqlalchemy.ext.automap.measurement object at 0x000002400C22B708>, 'USC00519397')
(<sqlalchemy.ext.automap.measurement object at 0x000002400C22B848>, 'USC00519397')
(<sqlalchemy.ext.automap.measurement object at 0x000002400C22B8C8>, 'USC00519397')
(<sqlalchemy.ext.automap.measurement object at 0x000002400C22B948>, 'USC00519397')
(<sqlalchemy.ext.automap.measurement object at 0x000002400C22B9C8>, 'USC00519397')


In [13]:
# Use the session to query station table and display the first 5 stations
for row in session.query(station, station.station).limit(5).all():
    print(row)

(<sqlalchemy.ext.automap.station object at 0x000002400C238148>, 'USC00519397')
(<sqlalchemy.ext.automap.station object at 0x000002400C238288>, 'USC00513117')
(<sqlalchemy.ext.automap.station object at 0x000002400C238308>, 'USC00514830')
(<sqlalchemy.ext.automap.station object at 0x000002400C238388>, 'USC00517948')
(<sqlalchemy.ext.automap.station object at 0x000002400C238408>, 'USC00518838')


# Exploratory Precipitation Analysis

In [21]:
# Find the most recent date in the data set in measurment table.  Note station table has no date data.
recent = session.query(measurement.date).order_by(measurement.date.desc()).first()
print(recent)

('2017-08-23',)


In [34]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
print(dt.date(2017, 8 ,23))
# Calculate the date one year from the last date in data set.
year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(year_ago)


2017-08-23
2016-08-23


In [37]:
# Perform a query to retrieve the data and precipitation scores
precipitation = [measurement.id,
                measurement.station,
                measurement.date,
                measurement.prcp]

prcp_score = session.query(*precipitation).\
                    filter(measurement.date > '2016-08-23').\
                    filter(measurement.date < '2017-08-23').\
                    order_by(measurement.date).all()
                    
# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data


[(2365, 'USC00519397', '2016-08-24', 0.08),
 (5092, 'USC00513117', '2016-08-24', 2.15),
 (7332, 'USC00514830', '2016-08-24', 2.28),
 (8826, 'USC00517948', '2016-08-24', None),
 (11852, 'USC00519523', '2016-08-24', 1.22),
 (14609, 'USC00519281', '2016-08-24', 2.15),
 (19203, 'USC00516128', '2016-08-24', 1.45),
 (2366, 'USC00519397', '2016-08-25', 0.08),
 (5093, 'USC00513117', '2016-08-25', 0.08),
 (7333, 'USC00514830', '2016-08-25', 0.0),
 (8827, 'USC00517948', '2016-08-25', 0.0),
 (11853, 'USC00519523', '2016-08-25', 0.21),
 (14610, 'USC00519281', '2016-08-25', 0.06),
 (19204, 'USC00516128', '2016-08-25', 0.11),
 (2367, 'USC00519397', '2016-08-26', 0.0),
 (5094, 'USC00513117', '2016-08-26', 0.03),
 (7334, 'USC00514830', '2016-08-26', 0.02),
 (8828, 'USC00517948', '2016-08-26', 0.04),
 (11854, 'USC00519523', '2016-08-26', 0.0),
 (14611, 'USC00519281', '2016-08-26', 0.01),
 (2368, 'USC00519397', '2016-08-27', 0.0),
 (5095, 'USC00513117', '2016-08-27', 0.18),
 (7335, 'USC00514830', '2016-

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()